In [5]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Input, LeakyReLU
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)  # 保持原始图像形状并添加通道维度
x_test = x_test.reshape(-1, 28, 28, 1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.2,
    shear_range=0.15,
    fill_mode="nearest",
)
datagen.fit(x_train)

# 构建改进后的BP神经网络模型
model = Sequential(
    [
        Input(shape=(28, 28, 1)),  # 使用Input层指定输入形状
        Flatten(),  # 展平图像

        Dense(2048),  # 输入层 -> 隐藏层 1
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(1024),  # 隐藏层 2
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(512),  # 隐藏层 3
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(256),  # 隐藏层 4
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(128),  # 隐藏层 5
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(64),  # 隐藏层 6
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(32),  # 隐藏层 7
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(10, activation="softmax"),  # 输出层
    ]
)

# 编译模型
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),  # 使用RMSprop优化器
    loss="categorical_crossentropy",  # 多分类交叉熵损失
    metrics=["accuracy"],
)

# 回调函数
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=3, min_lr=0.00001
)

early_stopping = EarlyStopping(
    monitor="val_loss", patience=8, restore_best_weights=True
)

# 训练模型
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=128),  # 调整批量大小
    epochs=300,  # 增加训练轮数
    validation_data=(x_test, y_test),
    callbacks=[reduce_lr, early_stopping],
)

# 测试模型
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# 保存模型
model.save("mnist_model_bp_v1.keras")

Epoch 1/300


/Users/xinbob/PycharmProjects/tensorflow-sample/env/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


469/469 ━━━━━━━━━━━━━━━━━━━━ 30s 55ms/step - accuracy: 0.2212 - loss: 2.4637 - val_accuracy: 0.7574 - val_loss: 0.6574 - learning_rate: 0.0010
Epoch 2/300
469/469 ━━━━━━━━━━━━━━━━━━━━ 25s 52ms/step - accuracy: 0.6414 - loss: 1.0683 - val_accuracy: 0.8720 - val_loss: 0.3988 - learning_rate: 0.0010
Epoch 3/300
469/469 ━━━━━━━━━━━━━━━━━━━━ 25s 53ms/step - accuracy: 0.7357 - loss: 0.8397 - val_accuracy: 0.9220 - val_loss: 0.3034 - learning_rate: 0.0010
Epoch 4/300
469/469 ━━━━━━━━━━━━━━━━━━━━ 25s 52ms/step - accuracy: 0.7881 - loss: 0.7267 - val_accuracy: 0.9506 - val_loss: 0.1900 - learning_rate: 0.0010
Epoch 5/300
469/469 ━━━━━━━━━━━━━━━━━━━━ 25s 54ms/step - accuracy: 0.8216 - loss: 0.6519 - val_accuracy: 0.9542 - val_loss: 0.1693 - learning_rate: 0.0010
Epoch 6/300
469/469 ━━━━━━━━━━━━━━━━━━━━ 25s 54ms/step - accuracy: 0.8495 - loss: 0.5741 - val_accuracy: 0.9555 - val_loss: 0.1600 - learning_rate: 0.0010
Epoch 7/300
469/469 ━━━━━━━━━━━━━━━━━━━━ 25s 54ms/step - accuracy: 0.8608 - loss: 